# Tarea corta 01

Machine Learning - Maestría de Ciencias de la computación
Profesor Dr. Jose Carranza Rojas

Estudiantes:
+ Jason Carmona (20NN...)
+ Edisson López (2013103311)

## Usando listas

In [2]:
""" Code of Jason Carmona """


## Usando Pytorch

In [59]:
"""Code of Edisson López"""




--- última línea ---